## Notebook #2: Initial Models and Null Model

In this notebook, I will form my first simple models including a basic linear regression based on the initial relationships I found in my cleaning process. As I explained in the first notebook, I did not realize I shouldn't have dropped those high NA columns until later, so you will see that they have been dropped again in this notebook. I will also evaluate the null model to get a baseline for metrics moving forward.

My first model is a multiple linear regression with just 5 features and no pre-processing. This will allow me to examine relationships on a simplified level.  

---

In [378]:
# adding all used libraries to the top of every notebook

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LassoCV

**BASIC LINEAR MODEL:**

In [590]:
X = df_start.drop(columns=['saleprice'])  #using the df_start dataset I created in notebook #1
y = df_start['saleprice']

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42)

In [591]:
tr_id = df_house['id']
val_id = df_house['id']

In [592]:
X_train.shape

(1536, 5)

In [593]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [594]:
lr.score(X_train, y_train), lr.score(X_val, y_val)

(0.7840044598338779, 0.8014371676336479)

As we can see here, the initial model didn't do horribly actually! But it did not perform as well as it could. 78-80% of the variation in our data can be explained by this first model; however, that $R^2$ can be improved. Let's look at some more metrics.

In [595]:
list(zip(X_train.columns, lr.coef_))

[('year_remod/add', 398.02332980492326),
 ('gr_liv_area', 64.52784582745593),
 ('totrms_abvgrd', -2815.5170684130785),
 ('overall_qual', 23712.718046978935),
 ('garage_area', 71.17705835488985)]

In [596]:
cross_val_score(lr, X, y, cv = 10).mean()

0.788632862255539

What is encouraging about this model is that the $R^2$ for all evaluations (X_train, X_val, and cross val) are around the same number. This shows that there is a good bias-variance trade off in our model and data. 

In [597]:
y_train_preds = lr.predict(X_train)

In [599]:
mean_squared_error(y_train, y_train_preds)

1354789141.6102898

In [600]:
y_val_preds = lr.predict(X_val)

In [601]:
mean_squared_error(y_val, y_val_preds)

1255336973.9312356

Whew! These are some very high mean squared error scores (should be as close to 0 as possible). To have a better understanding of these numbers, a look at the null model is necessary. 

--- 

**NULL MODEL:** 

In [375]:
y_baseline_preds = np.full_like(y, y.mean())

In [376]:
y_baseline_preds

array([181643, 181643, 181643, ..., 181643, 181643, 181643])

In [587]:
mean_squared_error(y, y_baseline_preds)

6284773122.026842

From calculating the mean squared error of the null model, we can see that even the initial linear regression model would be a better fit for the data than just predicting the mean for each data point (what the null model is doing). As I continue to develop my models, I will refer back to this null model mean squared error score to ensure I am not losing track of a well fitting model.

--- 

Next, I will put my initial linear regression model into a Lasso CV regularization and add in all numeric variables. In the end, I want to be able to isolate relevant features from the data. As I build my model, I will continue to add in and take out features to see what best fits the data. I am choosing to start with Lasso CV because it would help me truly isolate those best variables due to the absolute zero penalty within the function. Further, I saw some multicolinearity between some features, and Lasso CV can help wrangle that to something that is workable by decreasing the variance. I am choosing the CV version over the plain Lasso regularization because I want to add the extra testing of cross validation within my model. I will use these scores continually throughout this project to see how my model is doing to explain the variation in the data.

---

For the first Lasso CV model I made, I only wanted the numeric columns from the data. Therefore, I created a new dataset called df_nums containing only those features, as seen below. This allowed me to isolate only the columns I wanted to test. This dataset will continue to be used throughout the modeling process. The df_start dataset will no longer be used.

In [573]:
df_nums = df_house.select_dtypes(include=[np.number])
df_nums.head(3)

,id,ms_subclass,lot_frontage,lot_area,overall_qual,overall_cond,year_built,year_remod/add,mas_vnr_area,bsmtfin_sf_1,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,1st_flr_sf,2nd_flr_sf,low_qual_fin_sf,gr_liv_area,bsmt_full_bath,bsmt_half_bath,full_bath,half_bath,bedroom_abvgr,kitchen_abvgr,totrms_abvgrd,fireplaces,garage_yr_blt,garage_cars,garage_area,wood_deck_sf,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,misc_val,mo_sold,yr_sold,saleprice
0,109,60,NaN,13517,6,8,1976,2005,289.0,533.0,0.0,192.0,725.0,725,754,0,1479,0.0,0.0,2,1,3,1,6,0,1976.0,2.0,475.0,0,44,0,0,0,0,0,3,2010,130500
1,544,60,43.0,11492,7,5,1996,1997,132.0,637.0,0.0,276.0,913.0,913,1209,0,2122,1.0,0.0,2,1,4,1,8,1,1997.0,2.0,559.0,0,74,0,0,0,0,0,4,2009,220000
2,153,20,68.0,7922,5,7,1953,2007,0.0,731.0,0.0,326.0,1057.0,1057,0,0,1057,1.0,0.0,1,0,3,1,5,0,1953.0,1.0,246.0,0,52,0,0,0,0,0,1,2010,109000


In [574]:
house_features = [col for col in df_nums.columns if col != 'saleprice' and 'id']  # list comp instead of dropping the columns by hand

X = df_nums[house_features].drop(columns='id')
y = df_nums['saleprice']

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42)

As I worked through the first few models, I made many mistakes. I discussed one of those in my first notebook (dropping columns from misunderstanding the data description). Another mistake I made was imputing the missing values with the mean instead of 0 constant. I filled the values with the mean because, again, I did not read or understand the data description fully before beginning my modeling process. I have definitely learned that lesson! However, I wanted to show my honest process, so I am keeping my initial mistakes in here. Later, I will fix this to replace the missing values with 0 as the NA in the data description does not mean those data points are missing - just a 0 in the category for the variable.

In [576]:
si = SimpleImputer(strategy = 'mean')  #replacing missing values

X_train_fill = si.fit_transform(X_train)
X_val_fill = si.transform(X_val)

In [577]:
X_train_fill = pd.DataFrame(X_train_fill, columns = si.feature_names_in_)
X_val_fill = pd.DataFrame(X_val_fill, columns = si.feature_names_in_)

In [578]:
ss = StandardScaler()  # scaling the data before regularization
X_train_sc = ss.fit_transform(X_train_fill)
X_val_sc = ss.transform(X_val_fill)

X_train_sc = pd.DataFrame(X_train_sc, columns = ss.get_feature_names_out())
X_val_sc = pd.DataFrame(X_val_sc, columns = ss.get_feature_names_out())

In [579]:
lasso_cv = LassoCV(cv = 10).fit(X_train_sc, y_train)  # using 10 cross val k-folds for robust validation process

print('best alpha:', lasso_cv.alpha_)
print('score:', lasso_cv.score(X_train_sc, y_train))

best alpha: 274.5271935073856
score: 0.8712124957532725


In [580]:
new_lasso = Lasso(alpha = lasso_cv.alpha_)
new_lasso.fit(X_train_sc, y_train)

final_columns = [col for col, coef in zip(X_train_sc.columns, new_lasso.coef_) if coef]

In [377]:
lr = LinearRegression()
cross_val_score(lr, X_train_sc[final_columns], y_train, cv=10, n_jobs = -1).mean()

0.9029829428704517

Here, I ran the Lasso CV to get the best alpha. Then, I ran that alpha back through a regular Lasso model along with the best features from that model to see what would happen. It turned out that my model performed better when it just went through the Lasso CV fit rather than the second round of regularization. This is the first and final time I tried this. As you can see, my $R^2$ scores, again, are not terrible. I want to keep improving though. This is the first model I submitted to Kaggle (final Kaggle submission on notebook #5). It scored around 31,000. 

---

In an attempt to simply try many different things within my model, I next tried adding or taking away PolynomialFeatures, RFE Feature Selection, RidgeCV, and ElasticNetCV. Polynomial Features ended up working for my Kaggle model, but unfortunately, a model with almost 2000 more features than it has data points is not something that I would send to production. The issue with that is the high variance of having so many added features, and not enough data points. Regularization and RFE Feature Selection should bring this down, but it ultimately led to me having a way overfit model to the train set, and very underpreforming on the test set. The model built below has some of those things I added and took away in commented form. Most other things are the same, but you can see where I would be able to add a code block back in as a test.

Along with adding and taking away transformers, I also messed around with the features I would add vs take away. At one point, I added all features in the data set, did PolynomialFeatures, and ended up with over 11,000 features and about 1,500 data points. Obviously, that was a very badly performing model. But, in my draw to learn, I loved trying out anything I could think of. I submitted variations of this model to Kaggle before landing on the final one in notebook #5.

In [61]:
X = df_combined.drop(columns=['id', 'saleprice'])
y = np.log(df_combined['saleprice'])

In [62]:
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42)

In [63]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [64]:
from sklearn.compose import make_column_transformer
ohe = OneHotEncoder(handle_unknown='ignore')
smart_encoder = make_column_transformer((ohe,
                                        ['neighborhood', 'house_style', 'exter_qual', 'kitchen_qual']),
                                        remainder='passthrough',
                                        verbose_feature_names_out=False)

X_train_enc = smart_encoder.fit_transform(X_train)
X_val_enc = smart_encoder.transform(X_val)

X_train_enc = pd.DataFrame(X_train_enc, columns = smart_encoder.get_feature_names_out())
X_val_enc = pd.DataFrame(X_val_enc, columns = smart_encoder.get_feature_names_out())

In [65]:
si = SimpleImputer(strategy = 'mean')  # at this point, I still had not realized my mistake with the mean vs constant 0


X_train_fill = si.fit_transform(X_train_enc)
X_val_fill = si.transform(X_val_enc)

In [66]:
X_train_fill = pd.DataFrame(X_train_fill, columns = si.feature_names_in_)
X_val_fill = pd.DataFrame(X_val_fill, columns = si.feature_names_in_)

In [67]:
ss = StandardScaler()
X_train_sc = ss.fit_transform(X_train_fill)
X_val_sc = ss.transform(X_val_fill)

X_train_sc = pd.DataFrame(X_train_sc, columns = ss.get_feature_names_out())
X_val_sc = pd.DataFrame(X_val_sc, columns = ss.get_feature_names_out())

In [68]:
X_train_sc.shape

(1536, 80)

In [69]:
# from sklearn.preprocessing import PolynomialFeatures
# poly = PolynomialFeatures(include_bias=False)
# X_train_p = poly.fit_transform(X_train_sc)
# X_val_p = poly.transform(X_val_sc)

# X_train_p = pd.DataFrame(X_train_p, columns = poly.get_feature_names_out())
# X_val_p = pd.DataFrame(X_val_p, columns = poly.get_feature_names_out())

In [70]:
# from sklearn.feature_selection import RFE
# rfe = RFE(estimator=LinearRegression())
# X_train_half = rfe.fit_transform(X_train_p, y_train)
# X_val_half = rfe.transform(X_val_p)
# X_train_half = pd.DataFrame(X_train_half, columns = rfe.get_feature_names_out())
# X_val_half = pd.DataFrame(X_val_half, columns = rfe.get_feature_names_out())

In [71]:
X_train_sc.shape

(1536, 80)

In [72]:
from sklearn.linear_model import RidgeCV

rg_cv = RidgeCV(cv= 10).fit(X_train_sc, y_train)

print('best alpha:', rg_cv.alpha_)
print('score:', rg_cv.score(X_train_sc, y_train))

best alpha: 10.0
score: 0.9190596946095148


In [73]:
# from sklearn.linear_model import ElasticNetCV

# elastic_cv = ElasticNetCV(cv= 10, n_jobs=-1).fit(X_train_sc, y_train)

# print('best alpha:', elastic_cv.alpha_)
# print('score:', elastic_cv.score(X_train_sc, y_train))

In [74]:
rg_cv.score(X_val_sc, y_val)

0.8930552952169875

The last model I tried as a part of this exploration finally introduced the steps that led to my best model. I remembered that, when I did my initial analysis, I graphed a histogram of sale price. Initially, it looked a little skewed, so I applied a log transformation, and that made the distribution more normal. I found a way to add this to my model using TransformedTargetRegressor:

In [34]:
X = df_combined.drop(columns=['id', 'saleprice'])
y = df_combined['saleprice']

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42)

In [36]:
ohe = OneHotEncoder(handle_unknown='ignore')
smart_encoder = make_column_transformer((ohe,
                                        ['neighborhood', 'house_style', 'exter_qual', 'kitchen_qual']),
                                        remainder='passthrough',
                                        verbose_feature_names_out=False)

X_train_enc = smart_encoder.fit_transform(X_train)
X_val_enc = smart_encoder.transform(X_val)

X_train_enc = pd.DataFrame(X_train_enc, columns = smart_encoder.get_feature_names_out())
X_val_enc = pd.DataFrame(X_val_enc, columns = smart_encoder.get_feature_names_out())

In [37]:
si = SimpleImputer(strategy = 'mean')
# si = SimpleImputer(strategy = 'constant', fill_value=0)

X_train_fill = si.fit_transform(X_train_enc)
X_val_fill = si.transform(X_val_enc)

In [38]:
X_train_fill = pd.DataFrame(X_train_fill, columns = si.feature_names_in_)
X_val_fill = pd.DataFrame(X_val_fill, columns = si.feature_names_in_)

In [39]:
ss = StandardScaler()
X_train_sc = ss.fit_transform(X_train_fill)
X_val_sc = ss.transform(X_val_fill)

X_train_sc = pd.DataFrame(X_train_sc, columns = ss.get_feature_names_out())
X_val_sc = pd.DataFrame(X_val_sc, columns = ss.get_feature_names_out())

In [40]:
from sklearn.linear_model import LassoCV

lasso_cv = LassoCV(cv = 10).fit(X_train_sc, y_train)

print('best alpha:', lasso_cv.alpha_)
print('score:', lasso_cv.score(X_train_sc, y_train))

best alpha: 294.3665111484536
score: 0.9147164487058445


In [41]:
lasso_cv.score(X_val_sc, y_val)

0.9209348090028185

In [42]:
from sklearn.compose import TransformedTargetRegressor
tt = TransformedTargetRegressor(regressor = lasso_cv, func = np.log, inverse_func = np.exp)

In [43]:
tt.fit(X_train_sc, y_train)

TransformedTargetRegressor(func=<ufunc 'log'>, inverse_func=<ufunc 'exp'>,
                           regressor=LassoCV(cv=10))

In [44]:
tt.score(X_train_sc, y_train)

0.9348260075279748

In [45]:
tt.score(X_val_sc, y_val)

0.9359349820822778

In [46]:
col_coefs = lasso_cv.coef_

In [47]:
[col for col,coef in list(zip(X_train_sc.columns, col_coefs)) if coef > 0]

['neighborhood_Blmngtn',
 'neighborhood_BrkSide',
 'neighborhood_ClearCr',
 'neighborhood_Crawfor',
 'neighborhood_GrnHill',
 'neighborhood_NPkVill',
 'neighborhood_NoRidge',
 'neighborhood_NridgHt',
 'neighborhood_SWISU',
 'neighborhood_Somerst',
 'neighborhood_StoneBr',
 'neighborhood_Timber',
 'house_style_1.5Unf',
 'house_style_2.5Fin',
 'house_style_2.5Unf',
 'house_style_SFoyer',
 'house_style_SLvl',
 'exter_qual_Ex',
 'kitchen_qual_Ex',
 'lot_frontage',
 'lot_area',
 'overall_qual',
 'overall_cond',
 'year_built',
 'year_remod/add',
 'mas_vnr_area',
 'bsmtfin_sf_1',
 'bsmtfin_sf_2',
 'total_bsmt_sf',
 '2nd_flr_sf',
 'gr_liv_area',
 'bsmt_full_bath',
 'full_bath',
 'half_bath',
 'totrms_abvgrd',
 'fireplaces',
 'garage_yr_blt',
 'garage_cars',
 'garage_area',
 'wood_deck_sf',
 'open_porch_sf',
 'enclosed_porch',
 '3ssn_porch',
 'screen_porch']

In [187]:
list(zip(X_train_sc.columns, col_coefs))

[('neighborhood_Blmngtn', 429.89320651687893),
 ('neighborhood_Blueste', 0.0),
 ('neighborhood_BrDale', -0.0),
 ('neighborhood_BrkSide', 1253.6533894337874),
 ('neighborhood_ClearCr', 510.396871948398),
 ('neighborhood_CollgCr', 0.0),
 ('neighborhood_Crawfor', 3220.6544520134767),
 ('neighborhood_Edwards', -0.0),
 ('neighborhood_Gilbert', -150.9269357624419),
 ('neighborhood_Greens', 0.0),
 ('neighborhood_GrnHill', 2691.865215522986),
 ('neighborhood_IDOTRR', 0.0),
 ('neighborhood_Landmrk', 0.0),
 ('neighborhood_MeadowV', -0.0),
 ('neighborhood_Mitchel', -16.25815213341542),
 ('neighborhood_NAmes', -938.1657802376317),
 ('neighborhood_NPkVill', 123.80377586837243),
 ('neighborhood_NWAmes', -2478.279666898217),
 ('neighborhood_NoRidge', 4360.1458793303655),
 ('neighborhood_NridgHt', 6116.405220064707),
 ('neighborhood_OldTown', -175.56948564336201),
 ('neighborhood_SWISU', 415.32312320180387),
 ('neighborhood_Sawyer', -441.9637122737224),
 ('neighborhood_SawyerW', -928.489816488109),
 (

In [167]:
y_train_preds = tt.predict(X_train_sc)

In [168]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_train, y_train_preds)

408790928.037594

In [170]:
y_val_preds = tt.predict(X_val_sc)
mean_squared_error(y_val, y_val_preds)

405026382.67821234

In this notebook, I explored many different model styles and transformers to find what worked. I ultimately learned that Ridge and ElasticNet did not do enough for my model, yet Lasso CV seemed to work really well. I assume this is because my model has many features. Additionally, I eventually learned that I needed to change my imputer to use the constant 0 instead of the mean, and that I needed to not drop those initial columns just because they had a lot of NA values. Both of these things came with spending time in the data and learning how to set up initial modeling inputs. The best MSE score I got from these temporary models was in the 408,000,000 (408 million) range, which is far better than the null model MSE of around 6,200,000,000 (6.2 billion). 

---

In the next notebook, I will present my final model that has worked the best for me. It is able to explain my problem statement well and provide the insights I was hoping to explore. 